In [1]:
# For Earth Engine
import ee
import geemap

# For Vector Operations
import geopandas as gpd

# For File Operations
import os
import glob

import shapely
from shapely.geometry import Polygon

# from ee_computation import flatten_geom, s2_collection

In [2]:
# ee.Authenticate()
ee.Initialize()
Map = geemap.Map()

In [3]:
def flatten_geom(geometry) -> shapely.geometry.Polygon:
    '''
    Takes the Geometry in 3D and Returns 2D Geometry
    
    Input: Shapely.Geometry.Polygon
    Output: list of Coordinates
    '''
    poly_wkt = geometry.exterior
    flat_geom = list(Polygon([xy[0:2] for xy in list(poly_wkt.coords)]).exterior.coords)
    return flat_geom

def NDWIS2(image):
    ndwi = image.normalizedDifference(['B8','B12']).rename('NDWI')
    return image.addBands(ndwi)


def NDVIS2(image):
    ndvi = image.normalizedDifference(['B8','B4']).rename('NDVI')
    return image.addBands(ndvi)

def sclCloudMask_ndvi(image):
    ndvi = image.select('NDVI')
    crs = (ndvi.projection()).crs()
    scl = image.select('SCL')
    reScl = scl.resample('bilinear').reproject(crs = crs, scale = 10)
    mask = reScl.gt(3) and reScl.lte(7)
    maskedNdvi = ndvi.updateMask(mask)
    return ee.Image(maskedNdvi)


def sclCloudMask_ndwi(image):
    ndvi = image.select('NDWI')
    crs = (ndvi.projection()).crs()
    scl = image.select('SCL')
    reScl = scl.resample('bilinear').reproject(crs = crs, scale = 10)
    mask = reScl.gt(3) and reScl.lte(7)
    maskedNdvi = ndvi.updateMask(mask)
    return ee.Image(maskedNdvi)


def clipImage(image):
    return image.clip(aoi)

def getIndexRaster(aoi,sdate,edate, indice):
    if indice == "NDVI":
        dataset = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(sdate, edate).filterBounds(aoi)
        data=dataset.map(NDVIS2).map(sclCloudMask_ndvi).select('NDVI').map(clipImage).mean()
        geemap.ee_export_image(data, f'{indice}_image.tif', scale = 10, region = aoi)
    else:
        dataset = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(sdate, edate).filterBounds(aoi)
        data=dataset.map(NDWIS2).map(sclCloudMask_ndwi).select('NDWI').map(clipImage).mean()
        geemap.ee_export_image(data, f'{indice}_image.tif', scale = 10, region = aoi)


In [4]:

shapefile_path = "AOI/Kalahandi_AOI.shp"

gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs(4326)

flat_geometry = flatten_geom(gdf.geometry[0])

aoi = ee.Geometry.Polygon(flat_geometry)

sdate = "2022-01-01"
edate = "2022-12-01"
# foo = getIndexRaster(aoi,sdate,edate, "NDVI")


-------

## NDVI NDWI Using Landsat

In [22]:
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

# # Function to calculate NDWI
def calculate_ndwi(image):
    ndwi = image.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDWI')
    return image.addBands(ndwi)


landsat9 = (
    # ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA')
    ee.ImageCollection("LANDSAT/LC09/C02/T1_L2") 
    .filterDate(sdate, edate)
    .filterBounds(aoi)
).map(calculate_ndvi).map(calculate_ndwi)

ndvi_image = landsat9.select('NDVI').median().clip(aoi)
ndwi_image = landsat9.select('NDWI').median().clip(aoi)
# geemap.ee_export_image(foo, "landsat_ndwi.tif")

In [23]:
task = ee.batch.Export.image.toDrive(
    image=ndwi_image,
    description='ndwi_export',
    folder='ee_demos',
    region=aoi,
    scale=30,
    crs='EPSG:4326'
)

task.start()

------
## NDVI and NDWI using MODIS 

In [16]:
modis = ee.ImageCollection('MODIS/006/MOD09GA') \
    .filterDate('2022-01-01', '2022-01-30') \
    .filterBounds(aoi) \
    .select(['sur_refl_b01', 'sur_refl_b02'])  # bands 1 (red) and 2 (near-infrared)

# Define the NDVI calculation function
def calculate_ndvi_modis(image):
    ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('ndvi')
    return image.addBands(ndvi)

def calculate_ndwi_modis(image):
    ndwi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b03']).rename('ndwi')
    return image.addBands(ndwi)

In [17]:
modis_ndvi = modis.map(calculate_ndvi_modis).mean().clip(aoi)

In [18]:
task = ee.batch.Export.image.toDrive(
    image=modis_ndvi,
    description='modis_ndvi',
    folder='ee_demos',
    region=aoi,
    scale=250,
    crs='EPSG:4326'
)

task.start()

In [24]:
Map.centerObject(aoi, 10)
Map.addLayer(ndwi_image)
Map

Map(bottom=464880.0, center=[19.846209304392055, 82.82125549696741], controls=(WidgetControl(options=['positio…